Extrair *dataset*

In [28]:
!unzip tweetSentBR_extracted.zip
%cd tweetSentBR_extracted

Archive:  tweetSentBR_extracted.zip
   creating: tweetSentBR_extracted/
  inflating: tweetSentBR_extracted/testTT.neg  
  inflating: tweetSentBR_extracted/testTT.neu  
  inflating: tweetSentBR_extracted/testTT.pos  
  inflating: tweetSentBR_extracted/trainTT.neg  
  inflating: tweetSentBR_extracted/trainTT.neu  
  inflating: tweetSentBR_extracted/trainTT.pos  
  inflating: tweetSentBR_extracted/tweets.none  
  inflating: tweetSentBR_extracted/tweets.neg  
  inflating: tweetSentBR_extracted/tweets.neu  
  inflating: tweetSentBR_extracted/tweets.pos  
/content/tweetSentBR_extracted


Ler arquivos e separar por tab

In [31]:
import pandas as pd
import os
import re

corpus = [f for f in os.listdir(os.curdir) if 'tweets' in f and not 'tab' in f]

for filepath in corpus:
    with open(filepath) as f:
        text = f.read()

    assert '\t' not in text 
    
    outtext = re.sub(r'(.+?) (.+)',r'\1\t\2', text)
    outpath = re.sub(r'(.+)\.(.+)', r'\1_tab.\2',filepath)
    with open(outpath, 'w') as f:
        f.write(outtext)
        print(outpath)

tweets_tab.pos
tweets_tab.neu
tweets_tab.none
tweets_tab.neg


Gerar lista de sentenças para serem traduzidas

In [45]:
tabbed = [f for f in os.listdir(os.curdir) if 'tab' in f]
header = ['id', 'text']

sentences = set()
for f in tabbed:
    table = pd.read_csv(f, index_col = None, sep = '\t', names = header)
    sentences |= set(table['text'])

from pprint import pprint
pprint(sentences)

Streaming output truncated to the last 5000 lines.
 'uó',
 'parem de ficar cortando falas A pessoa não consegue terminar uma frase toda '
 'hr alguém atravessa sem or',
 'pareço normal mas cá estou revendo os episódios de só pra rever as cenas de '
 'vitor B e ana luiza 😂 🙈 shippo horrores',
 'participaria do só pelo vale carrefour de R $ 1000 tá foda fazer compras :/',
 'partiu assistir a reprise do e sofrer com a saida do USERNAME de novo pq sou '
 'dessas rever a cena q estraçalhou meu ❤ 😢 💔',
 'parto humanizado tem mulheres que têm coragem né',
 'parto normal e muita dor 😬 😬 😬',
 'pasmanter pra mim é o papel da vida dela maravilhosa ! ! #MaisVocê USERNAME',
 'pasmanterepivanomaisvocê começa logo porra chega de baboseira #MaisVoce',
 'pasquale amor a gente não vê preconceito em tudo o racismo está em tudo nas '
 'palavras e na nossa sociedade',
 'pasquale cipro neto no e falando verdades !',
 'pasquale maravilhoso #Encontro',
 'passa esse clipe logo pelo amor de deus #DomingoLegal',

Definir o código de tradução

In [0]:
from google.cloud import translate_v2 as translate
from os.path import isfile
from os import environ
import six
import json

def translate2dict(sentences, dictpath):
    environ['GOOGLE_APPLICATION_CREDENTIALS'] = './key.json'
    translate_client = translate.Client()

    def translation(text):
        if isinstance(text, six.binary_type):
            text = text.decode('utf-8')
        result = translate_client.translate(text,'en')

        return result['translatedText']

    if not isfile(dictpath):
        with open(dictpath, 'w') as f:
            json.dump({}, f)

    for item in sentences:
        with open(dictpath) as f:
            translations = json.load(f)
        
        try:
            translations[item]
        except KeyError:
            translations[item] = translation(item)
            
            with open(dictpath, 'w+') as f:
                json.dump(translations, f)

Traduzir

In [0]:
dictpath = 'tweetsent-dic.json'
translate2dict(sentences, dictpath)